In [8]:
from process_bigraph import Process, ProcessTypes, Composite

import cobra
from cobra.io import load_model

core = ProcessTypes()


# Dynamic FBA Process

In [9]:
class DynamicFBA(Process):
    """
    Performs dynamic FBA.

    Parameters:
    - model: The metabolic model for the simulation.
    - kinetic_params: Kinetic parameters (Km and Vmax) for each substrate.
    - biomass_reaction: The identifier for the biomass reaction in the model.
    - substrate_update_reactions: A dictionary mapping substrates to their update reactions.
    - biomass_identifier: The identifier for biomass in the current state.
    """
    
    config_schema = {
        'model_file': 'string',
        'kinetic_params': 'map[tuple[float,float]]',
        'biomass_reaction': {
            '_type': 'string',
            '_default': 'Biomass_Ecoli_core'
        },
        'substrate_update_reactions': 'map[string]',
        'biomass_identifier': 'string',
    }
  
    def __init__(self, config, core):
        super().__init__(config, core)
        
        if not 'xml' in self.config['model_file']:
            # use the textbook model if no model file is provided
            self.model = load_model(self.config['model_file'])
        else:
            self.model = cobra.io.read_sbml_model(self.config['model_file'])
        
    def inputs(self):
        return {'substrates': 'map[float]'}
    
    def outputs(self):
        return {'substrates': 'map[float]'}
    
    # TODO -- can we just put the inputs/outputs directly in the function?
    def update(self, state, interval):
        substrates_input = state['substrates']
        
        for substrate, reaction_id in self.config['substrate_update_reactions'].items():
            Km, Vmax = self.config['kinetic_params'][substrate]
            substrate_concentration = substrates_input[substrate]
            uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
            self.model.reactions.get_by_id(reaction_id).lower_bound = -uptake_rate
    
        substrate_update = {}
    
        solution = self.model.optimize()
        if solution.status == 'optimal':
            current_biomass = substrates_input[self.config['biomass_identifier']]
            biomass_growth_rate = solution.fluxes[self.config['biomass_reaction']]
            substrate_update[self.config['biomass_identifier']] = biomass_growth_rate * current_biomass * interval
    
            for substrate, reaction_id in substrate_update_reactions.items():
                flux = solution.fluxes[reaction_id]
                substrate_update[substrate] = flux * current_biomass * interval   # TODO -- assert not negative?
        else:
            # Handle non-optimal solutions if necessary
            print('Non-optimal solution, skipping update')
            pass
        
        print(substrate_update)
    
        return {
            'substrates': substrate_update,
        }

core.register_process('DynamicFBA', DynamicFBA)

In [10]:
from process_bigraph.experiments.parameter_scan import RunProcess

# TODO -- this should be imported, or just part of Process?
def run_process(
        address, 
        config, 
        core, 
        initial_state, 
        observables, 
        timestep=1, 
        runtime=10
):
    config = {
        'process_address': address,
        'process_config': config,
        'observables': observables,
        'timestep': timestep,
        'runtime': runtime}
       
    run = RunProcess(config, core).composite
    run.update(initial_state, runtime)
    return run.gather_results()

In [11]:
config = {
    'model_file': 'textbook',
    'kinetic_params': {
        'glucose': (0.5, 2),
        'acetate': (0.5, 2)
    },
    'biomass_reaction': 'Biomass_Ecoli_core',
    'substrate_update_reactions': {
        'glucose': 'EX_glc__D_e',
        'acetate': 'EX_ac_e'
    },
    'biomass_identifier': 'biomass'
}

initial_state = {
    'substrates': {
        'biomass': 0.1,  # Initial biomass concentration
        'glucose': 20.0, # Initial glucose concentration
        'acetate': 0.0   # Initial acetate concentration
    }
}

results = run_process(
    address='local:DynamicFBA', 
    config=config, 
    core=core, 
    initial_state=initial_state,
    observables=[['substrates', 'glucose'], 
                 ['substrates', 'acetate'],
                 ['substrates', 'biomass']], 
    timestep=1, 
    runtime=100,
)

Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solution, skipping update
{}
Non-optimal solu

/Users/eranagmon/code/bigraph-schema/venv/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [12]:
results

{('emitter',): [{'time': 0.0,
   'substrates': {'glucose': 0.0, 'acetate': 0.0, 'biomass': 0.0}}]}

In [13]:
model.reactions.EX_o2_e.lower_bound = -2  # Limiting oxygen uptake
model.reactions.ATPM.lower_bound = 1     # Setting lower bound for ATP maintenance
model.reactions.ATPM.upper_bound = 1     # Setting upper bound for ATP maintenance

# Define initial conditions, kinetic parameters, and other necessary inputs
initial_conditions = {
    'biomass': 0.1,  # Initial biomass concentration
    'glucose': 20.0, # Initial glucose concentration
    'acetate': 0.0   # Initial acetate concentration
}
kinetic_params = {
    'glucose': (0.5, 2), # Km and Vmax for glucose
    'acetate': (0.5, 2)  # Km and Vmax for acetate
}
substrate_update_reactions = {
    'glucose': 'EX_glc__D_e',  # Exchange reaction ID for glucose
    'acetate': 'EX_ac_e'       # Exchange reaction ID for acetate
}

# simulation conditions
t_n = 100 # number of time points
dt = 0.5  # Time step, matching your setup
time_points = list(range(0, t_n))  # Simulation time points, matching your setup

# Make sure to adjust the perform_dfba function if needed to initialize result arrays with sufficient size
dfba_results = perform_dfba(
    model, 
    initial_conditions, 
    kinetic_params, 
    time_points, 
    'Biomass_Ecoli_core', 
    substrate_update_reactions, 
    dt,
    'biomass'
)

NameError: name 'model' is not defined